In [ ]:
#We have launched this NoteBook on Google Collab

In [ ]:
!pip install transformers
!pip install nlp
!pip install sentencepiece
!pip install emoji

In [ ]:
import io
from google.colab import drive, files
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
auth.authenticate_user()  # verify your account to read files which you have access to. Make sure you have permission to read the file!
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm

#gc = gspread.authorize(GoogleCredentials.get_application_default()) 
import transformers
from transformers import AutoTokenizer
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

In [ ]:
# Normalisation for BertTweet
from nltk.tokenize import TweetTokenizer
from emoji import demojize
import re

tokenizer = TweetTokenizer()

# https://huggingface.co/vinai/bertweet-base
def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):

    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())

In [ ]:
#upload file 
uploaded = files.upload()

In [ ]:
#read uploaded file
tweets = pd.read_csv("file_name")
tweets=tweets.rename(columns={"fulltext": "text"})
print(tweets.shape)
tweets.head(2)

In [ ]:
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader

class TweetDataSet(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        for key in self.encodings.keys():
            return len(self.encodings[key])
        #return len(self.labels)

def proba_to_category(row):
  #print(row)
  score_0, score_1 = row.iloc[0], row.iloc[1]

  if score_0 < 0.5 and score_1 >= 0.5:
    return 1
  else: return 0

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("device: {}".format(device))
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
model = AutoModelForSequenceClassification.from_pretrained("path_model").to(device)

In [ ]:
# truncation, padding = true ensures that all sentences are padded to the same length and are truncated to be no longer model's max input lengts
# => allows to feed batches of sequences 
tweets_encodings = tokenizer(tweets.text.map(normalizeTweet).values.tolist(), truncation=True, padding=True, return_tensors="pt").to(device)
tweetDataSet = TweetDataSet(tweets_encodings)

model.eval()
tweetsLoader = DataLoader(tweetDataSet, batch_size=16)
print("len tweetsLoader: {}".format(len(tweetsLoader)))

predicted = pd.Series()
for (i, batch) in enumerate(tweetsLoader): 
    if i % 2000 == 0 : print(i)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)
    proba = F.softmax(outputs[0]).detach().cpu().numpy()  # get probabilities from output
    predicted_labels = pd.DataFrame(proba).apply(proba_to_category, axis=1) # get predicted class (highest proba)
    predicted = predicted.append(predicted_labels, ignore_index=True)

print("predicted: {}".format(predicted.shape))
print(predicted.value_counts())

tweets["jokes"] = predicted.values
tweets_no_jokes = tweets[tweets["jokes"] == 0]
print("No jokes tweets: {}".format(tweets_no_jokes.shape))

del tweets_no_jokes["jokes"]
tweets_no_jokes.to_csv("saving_path")